In [1]:
import pandas as pd
import numpy as np
import os
os.chdir('/Users/lihaohan/Alpha_Research')
from Data.utils import DButils
from tqdm import tqdm


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
  

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/lihaohan/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
  

AttributeError: _ARRAY_API not found

In [2]:
db = DButils()

In [3]:
db.cursor.execute('select * from st')
st = db.cursor.fetchall()
st = pd.DataFrame(st, columns=['code', 'date', 'st'])
st.sort_values(by=['code', 'date'], inplace=True)

db.cursor.execute('select * from limit_status')
limit_status = db.cursor.fetchall()
limit_status = pd.DataFrame(limit_status, columns=['code', 'date', 'limit_status'])
limit_status.sort_values(by=['code', 'date'], inplace=True)

db.cursor.execute('select * from suspend')
suspend = db.cursor.fetchall()
suspend = pd.DataFrame(suspend, columns=['code', 'date', 'suspend'])
suspend.sort_values(by=['code', 'date'], inplace=True)

In [4]:
from tqdm import tqdm

In [5]:
data_path = '/Volumes/T7Shield/Alpha_Research/成交量分布因子'
files = os.listdir('/Volumes/T7Shield/Alpha_Research/成交量分布因子')

In [6]:
def filter_st(st, data):
    factor_name = data.columns[-1]
    data = pd.merge(data, st, on=['code', 'date'], how='left')
    data['tmp_factor'] = np.where(data['st'] == 0, np.nan, data[factor_name])
    return data[['code', 'date', 'tmp_factor']]

def filter_limit(limit_status, data):
    factor_name = data.columns[-1]
    data = pd.merge(data, limit_status, on=['code', 'date'], how='left')
    data['tmp_factor'] = np.where(data['limit_status'] == 0, np.nan, data[factor_name])
    return data[['code', 'date', 'tmp_factor']]

def filter_suspend(suspend, data):
    factor_name = data.columns[-1]
    data = pd.merge(data, suspend, on=['code', 'date'], how='left')
    data['tmp_factor'] = np.where(data['suspend'] == 0, np.nan, data[factor_name])
    return data[['code', 'date', 'tmp_factor']]

In [7]:
save_path = '/Volumes/T7Shield/Alpha_Research/tmp_factor'

for file in tqdm(files):
    if file.startswith('._'):
        continue
    try:
        factor = pd.read_pickle(os.path.join(data_path, file))
        factor['date'] = pd.to_datetime(factor['date']).dt.strftime('%Y-%m-%d')
        factor = filter_st(st, factor)
        factor = filter_limit(limit_status, factor)
        factor = filter_suspend(suspend, factor)
        factor.to_pickle(os.path.join(save_path, file))
    except:
        print(f'{file} failed')

 41%|████      | 9/22 [12:30<14:48, 68.33s/it] 

early_noon_vol_ratio.pkl failed


100%|██████████| 22/22 [16:25<00:00, 44.82s/it]


In [8]:
data_path = '/Volumes/T7Shield/Alpha_Research/时序信息因子'
files = os.listdir('/Volumes/T7Shield/Alpha_Research/时序信息因子')

save_path = '/Volumes/T7Shield/Alpha_Research/tmp_factor'

for file in tqdm(files):
    try:
        factor = pd.read_pickle(os.path.join(data_path, file))
        factor['date'] = pd.to_datetime(factor['date']).dt.strftime('%Y-%m-%d')
        factor = filter_st(st, factor)
        factor = filter_limit(limit_status, factor)
        factor = filter_suspend(suspend, factor)
        factor.to_pickle(os.path.join(save_path, file))
    except:
        print(f'{file} failed')

100%|██████████| 52/52 [35:05<00:00, 40.49s/it]   

._vol_foc.pkl failed
._rtn_foc.pkl failed
._foc_comb.pkl failed
._rtn_DW.pkl failed
._vol_DW.pkl failed
._DW_comb.pkl failed
._rtn_rho.pkl failed
._vol_rho.pkl failed
._rho_comb.pkl failed
._rtn_LBQ.pkl failed
._vol_LBQ.pkl failed
._LBQ_Comb.pkl failed
._highStdRtn_mean.pkl failed
._hpos.pkl failed
._tvr.pkl failed
._pvi.pkl failed
._varr.pkl failed
._pvcorr.pkl failed
._icp.pkl failed
._r1min_skew.pkl failed
._trendratio.pkl failed
._vov.pkl failed
._SM_1.pkl failed
._SM_2.pkl failed
._SM_3.pkl failed
._SM_4.pkl failed


In [9]:
data_path = '/Volumes/T7Shield/Alpha_Research/收益率分布因子'
files = os.listdir('/Volumes/T7Shield/Alpha_Research/收益率分布因子')

save_path = '/Volumes/T7Shield/Alpha_Research/tmp_factor'

for file in tqdm(files):
    try:
        factor = pd.read_pickle(os.path.join(data_path, file))
        factor['date'] = pd.to_datetime(factor['date']).dt.strftime('%Y-%m-%d')
        factor = filter_st(st, factor)
        factor = filter_limit(limit_status, factor)
        factor = filter_suspend(suspend, factor)
        factor.to_pickle(os.path.join(save_path, file))
    except:
        print(f'{file} failed')

100%|██████████| 32/32 [15:55<00:00, 29.86s/it]

._real_var.pkl failed
._rv_up.pkl failed
._rv_down.pkl failed
._rv_umd.pkl failed
._rtn_skew.pkl failed
._rtn_kurt.pkl failed
._rtn5_mean.pkl failed
._nos.pkl failed
._exRtn_minVal.pkl failed
._exRtn_minFre.pkl failed
._exRtn_maxVal.pkl failed
._exRtn_maxFre.pkl failed
._gmm_mean.pkl failed
._gmm_mean2wgt.pkl failed
._gmm_meandif.pkl failed
._gmm_meandif2wgtdif.pkl failed


In [10]:
data_path = '/Volumes/T7Shield/Alpha_Research/tmp_factor'
save_path = '/Volumes/T7Shield/Alpha_Research/done_factor'

In [14]:
a = pd.read_pickle('/Volumes/T7Shield/Alpha_Research/tmp_factor/logvol_skew.pkl')
a = a.dropna(subset=['tmp_factor'])
a

,code,date,tmp_factor
0,000001,2018-01-02,-11.052269
1,000001,2018-01-03,-11.518387
2,000001,2018-01-04,-10.796422
3,000001,2018-01-05,-10.654594
4,000001,2018-01-08,-12.186086
...,...,...,...
7513761,920128,2025-01-21,-1.603047
7513762,920128,2025-01-22,-1.650055
7513763,920128,2025-01-23,-1.127775
7513764,920128,2025-01-24,-1.241256


In [15]:
b = a.copy()

In [16]:
b['factor'] = b.groupby('code')['tmp_factor'].rolling(20).mean().values
b

,code,date,tmp_factor,factor
0,000001,2018-01-02,-11.052269,NaN
1,000001,2018-01-03,-11.518387,NaN
2,000001,2018-01-04,-10.796422,NaN
3,000001,2018-01-05,-10.654594,NaN
4,000001,2018-01-08,-12.186086,NaN
...,...,...,...,...
7513761,920128,2025-01-21,-1.603047,-2.791134
7513762,920128,2025-01-22,-1.650055,-2.711863
7513763,920128,2025-01-23,-1.127775,-2.638876
7513764,920128,2025-01-24,-1.241256,-2.550584


In [26]:
files = os.listdir(data_path)
for file in tqdm(files):
    if file.startswith('._'):
        continue
    try:
        factor = pd.read_pickle(os.path.join(data_path, file))
        factor = factor.dropna(subset=['tmp_factor'])
        factor['factor'] = factor.groupby('code')['tmp_factor'].rolling(20).mean().values
        factor = factor[['code', 'date', 'factor']]
        factor.to_pickle(os.path.join(save_path, file))
    except:
        print(f'{file} failed')

100%|██████████| 104/104 [06:55<00:00,  3.99s/it]


In [2]:
groups_num = 10
each_group = 10
l = list(
        map(
            lambda x, y: [x] * y,
            list(range(1, groups_num + 1)),
            [each_group] * groups_num,
        )
    )
l

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
 [6, 6, 6, 6, 6, 6, 6, 6, 6, 6],
 [7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
 [8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
 [9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
 [10, 10, 10, 10, 10, 10, 10, 10, 10, 10]]